In [2]:
import kagglehub
import shutil

import os

# Download latest version
path = kagglehub.dataset_download("aadyasingh55/impact-of-social-media-on-suicide-rates")

# Move dataset to the desired folder
destination = os.path.join("datasets")

shutil.move(path, destination)


print("Path to dataset files:", destination)

C:\Users\HomePC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 811/811 [00:00<00:00, 543kB/s]

Extracting files...


Error: Destination path 'datasets\1' already exists

In [8]:
import pandas as pd
import mysql.connector
import getpass #Import getpass to securely input the password

In [14]:
# Extract data
def extract_data(file_path):
    data = pd.read_csv(file_path)
    print("Columns in DataFrame:", data.columns)
    data.columns = data.columns.str.strip()  # Strip whitespace from column names
    return data

# Transform data
def transform_data(data):
    # Rename columns to match the database schema
    data.rename(columns={
        'Suicide Rate % change since 2010': 'suicide_rate_change',
        'Twitter user count % change since 2010': 'twitter_user_count_change',
        'Facebook user count % change since 2010': 'facebook_user_count_change'
    }, inplace=True)
    return data

# load data
def load_data(data):
    password = getpass.getpass("Enter your Mysql password: ")
    # Connect to the database
    connection = mysql.connector.connect(
        host='localhost',
        user='root',
        password=password,
        database='social_media_data'
    )
    
    cursor = connection.cursor()
    
    # Create table if it doesn't exist
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS social_media_suicide_rates (
        id INT AUTO_INCREMENT PRIMARY KEY,
        year INT NOT NULL,
        sex VARCHAR(10) NOT NULL,
        suicide_rate_change FLOAT NOT NULL,
        twitter_user_count_change FLOAT NOT NULL,
        facebook_user_count_change FLOAT NOT NULL
        )
    """)
    
    # Insert data into the table
    for index, row in data.iterrows():
        cursor.execute("INSERT INTO social_media_suicide_rates (year, sex, suicide_rate_change, twitter_user_count_change, facebook_user_count_change) VALUES (%s, %s, %s, %s, %s)", (row['year'], row['sex'], row['suicide_rate_change'], row['twitter_user_count_change'], row['facebook_user_count_change']))
    
    connection.commit()
    cursor.close()
    connection.close()
#main function to run etl process
def main():
    file_path = './datasets/1/social-media-impact-on-suicide-rates.csv'

    #execute ETL process
    data = extract_data(file_path)
    data = transform_data(data)
    load_data(data)

if __name__ == '__main__':
    main()

Columns in DataFrame: Index(['year', 'sex', 'Suicide Rate % change since 2010',
       'Twitter user count % change since 2010',
       'Facebook user count % change since 2010'],
      dtype='object')
